In [1]:
# import moduel and data 
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns

# 自用模組


def get_num_data(df):
    return df.select_dtypes("number")

def get_nonNum_data(df):
    return df.select_dtypes(exclude="number")

#                                  ==== category data 相關    ====  

#放入"全部"資料，與預測標的名稱  
# 回傳 None ,但於stdout上顯示各項指標
def catData_show_statistic(df,y_name):
    
    for i in df.select_dtypes(exclude="number"):
        miss = df.isna().mean(axis=0).to_frame("miss_rate").sort_values("miss_rate",ascending=False).transpose()
        print("==========================")
        print("欄位:",i)
        
        print("資料缺失率:",miss[i]["miss_rate"])
        
        
        counts = df[i].value_counts()
        print("共",len(counts),"種資料\n") 
        for c in counts.index:
            print(c, "數量:",counts[c])
            print("此類型的y平均數", df[y_name][df[i] == c].to_frame("1").mean()["1"] )
            print("此類型的y中位數", df[y_name][df[i] == c].to_frame("1").median()["1"] )
            
#放入"全部"資料，與預測標的名稱  
# 回傳 將類別資料改成其類別對y的中位數 (資料缺失填入0)  與 被更改的類別label-> value 的dir
#                                                      供接下來testing使用replace(dir)
def catData_encoding(df,y_name):
    replace_dir = {}
    for i in df.select_dtypes(exclude="number"):
        counts = df[i].value_counts()
        
        
        
        replace_dir[i] = {}
        
        for c in counts.index:
            replace_dir[i][c] = df[y_name][df[i] == c].to_frame("1").mean()["1"]

    return df.replace(replace_dir),replace_dir
        
        
        
        
        
#                                  ====   missing rate 相關   ====                    #

def show_miss_rate(df):
    miss = df.isna().mean(axis=0).to_frame("miss_rate")
    miss = miss[miss.miss_rate!=0].sort_values("miss_rate",ascending=False)
    sns.catplot(kind="bar",data=miss.transpose(),aspect=2,height=10)
    print(miss)
# 印出所有含有na或null的標籤(特徵)，並依缺失率高低排序 
# 同時畫出表格

def drop_miss_rate(df,threshold):
    miss = df.isna().mean(axis=0).to_frame("miss_rate")
    miss = miss[miss.miss_rate!=0].sort_values("miss_rate",ascending=False).transpose()
    
    miss_index = []
    for i in miss:
        if (miss[i]>threshold).bool():
            miss_index.append(i)
    return df.drop(miss_index , axis=1)
# 回傳丟掉高於threshold遺失率的欄位  
# 0 <= threshold <= 1



#   ===================================   for num data   ===================================  # 
#                                  ====   分布相關   ====                    #

    

def numData_show_statistic(df,y_name):
    print("比較標的:常態分佈 偏度與峰度皆為0")
    print("偏度很大=>極度右偏 ， 峰度很大 => 值集中")
    miss = df.isna().mean(axis=0).to_frame("miss_rate").sort_values("miss_rate",ascending=False).transpose()

    for i in df:
        print("==========================")
        print("欄位:",i)
        print("    ---     ")
        print("平均值",df[i].mean())
        print("標準差",df[i].std())
        print("偏度:",df[i].skew())
        print("峰度:",df[i].skew())
        print("    ---     ")
        print("資料缺失率:",miss[i]["miss_rate"])
        print("    ---     ")
        print("Pearson相關係數(共線程度):",df.corr(method='pearson')[y_name][i])
        print("Spearman相關係數(廣義關係):",df.corr(method='spearman')[y_name][i])
        print("Kendall相關係數(單調關係):",df.corr(method='kendall')[y_name][i])
# 照欄位順序依次印出基本統計數據

#如名，印出y_mean 與 y_std 關係，確認是否要取log  
#(如果無關=>不用調  正比就開根、正二次方就取log)
def show_y_mean_and_std_relation(df,y_name):
    temp = {}
    temp["std"] = []
    temp["mu"] = []
    
    print("Y偏度:",df[y_name].skew())
    print("Y峰度:",df[y_name].kurt())

    #pd.set_option('display.float_format', lambda x: '%.3f' % x)

    for i in range(len(df[y_name])):

        temp["mu"].append(df[y_name][:i+1].mean())
        temp["std"].append(df[y_name][:i+1].std()*train_y[:i+1].std()  )

    temp = pd.DataFrame(temp) 
    print(temp)

    #from matplotlib import pyplot as plt
    #f, ax = plt.subplots()
    #ax.ticklabel_format(style='plain', axis='both')
    sns.relplot(x="std", y="mu", data=temp)


def numData_show_detail(df,x_name,y_name):
    des = df.describe()
    
    print("===============與目標關係程度===============\n\n")
    print("Pearson相關係數(共線程度):",df.corr(method='pearson')[y_name][x_name])
    print("Spearman相關係數(單調關係):",df.corr(method='spearman')[y_name][x_name])
    print("Kendall相關係數(單調關係):",df.corr(method='kendall')[y_name][x_name])
    print("\n")
    print("================基本數值================\n\n")
    print(des[x_name])
    print("\n")
    print("X偏度:",df[x_name].skew())
    print("X峰度:",df[x_name].skew())
    print("================與目標關係圖================\n\n")
    sns.relplot(x=x_name, y=y_name, data=df)
    print("\n")
# 單欄位詳細資料

# 輸入想要的correlation係數絕對值門檻，指定方法
# 回傳高於門檻的 "index label"(list)  
def numData_corr_filter(df,y_name,threshold,kind="spearman"):
    train_x_label = []
    for i in df:
 
        if abs(df.corr(method=kind)[y_name][i]) > threshold :
             if i != y_name:
                train_x_label.append(i)
    return train_x_label


# 限制所有值在n個標準差之內
# 回傳限制完的結果(df)
def numData_outlier_limit(df ,y_name , n):
    
    #label_counter = 0
    #total_labels = len(df.columns)
    
    for i in df:
        
        
        if i!=y_name:
            mu = df[i].mean()
            std = df[i].std()
            for j in range(len(df[i])):
                if df[i][j]<mu :
                    df.loc[j,i] = max(df[i][j],mu-n*std)


                if df[i][j]>mu:
                    df.loc[j,i] = min(df[i][j],mu+n*std)
        #label_counter+=1
        #print("Dealing outlier (Lable):",label_counter,"\\",total_labels)
    return df

# ===================================   for data processing   =================================== 
def z_score(df):
    return (df-df.mean())/df.std()
# 回傳標準化z-score後的表格(df)

def log(df):
    logdf = np.log(df+1)
    logdf= logdf.replace([np.inf, -np.inf], np.nan)
    
    return logdf.fillna(0)
# 回傳log transform 結果，0保持在0

def shuffle_data(df): 
    return df.sample(frac=1).reset_index(drop=True)
# 隨機排序data

def split_data(df,len_train_data):
    len_all_data = len(df.index)
    
    s = pd.Series( [i for i in range(0,len_all_data-len_train_data)])
    train = all_data.loc[:len_train_data]

    test = all_data.loc[len_train_data:].set_index(s)
    
    return train,test
# 將data 切成train與test,並且給定長度


In [2]:
# get data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")





train_y = train["SalePrice"]


"""
# data preprocessing  for  
1.encoding cat data by median
2.nan remove (by drop and replace)
3.scaling (take z_score)
4.outlier  
"""
# encoding
(train_x,replace_dir) = catData_encoding(train,"SalePrice")
# drop too many nan column 
train_x = drop_miss_rate(train_x , 0.2)
# nan remove
train_x = train_x.fillna(0)
# outlier 
train_x = numData_outlier_limit(train_x,"SalePrice",3)
print(train_x)
# scaling 
train_x = z_score(train_x)

# correlation
train_x_label = numData_corr_filter(train_x ,"SalePrice",0.3)



#train_x


train_x = train_x[train_x_label]


#temp["SalePrice"] = train_y
#numData_show_detail(temp,"3SsnPorch","SalePrice")
#print(train_x_corr["3SsnPorch"])

        Id  MSSubClass       MSZoning  LotFrontage  LotArea         Street  \
0        1        60.0  191004.994787         65.0   8450.0  181130.538514   
1        2        20.0  191004.994787         80.0   9600.0  181130.538514   
2        3        60.0  191004.994787         68.0  11250.0  181130.538514   
3        4        70.0  191004.994787         60.0   9550.0  181130.538514   
4        5        60.0  191004.994787         84.0  14260.0  181130.538514   
...    ...         ...            ...          ...      ...            ...   
1455  1456        60.0  191004.994787         62.0   7917.0  181130.538514   
1456  1457        20.0  191004.994787         85.0  13175.0  181130.538514   
1457  1458        70.0  191004.994787         66.0   9042.0  181130.538514   
1458  1459        20.0  191004.994787         68.0   9717.0  181130.538514   
1459  1460        20.0  191004.994787         75.0   9937.0  181130.538514   

           LotShape    LandContour     Utilities      LotConfig

In [10]:
import sklearn

#from sklearn.linear_model import Ridge
#reg = Ridge(alpha=0.2)
#from sklearn import linear_model
#reg = linear_model.LinearRegression()
#from sklearn.linear_model import LogisticRegression
#reg = LogisticRegression(max_iter=2000)
#from sklearn import linear_model
#reg = linear_model.ARDRegression()
#from sklearn.ensemble import RandomForestRegressor
#reg = RandomForestRegressor(max_depth=4, random_state=0)
#from sklearn.ensemble import GradientBoostingRegressor
#reg = GradientBoostingRegressor(n_estimators=2048, learning_rate=0.01,\
#max_depth=2, random_state=0, loss='ls')

from sklearn.neural_network import MLPRegressor
reg = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=16,max_iter=4000,alpha=1,verbose=True ,learning_rate="adaptive")
#from sklearn import svm
#reg=svm.SVR()
#from sklearn import tree
#reg = tree.DecisionTreeRegressor()
#reg = sklearn.neural_network.MLPRegressor(tol=1e-7,learning_rate_init=0.06428,hidden_layer_sizes=(60,30,15,7,3,1),max_iter=15000,verbose=True)
#print(train_x)
#reg.fit(train_x, np.sqrt(train_y))
reg.fit(train_x, np.log(train_y))

Iteration 1, loss = 81.86023283
Iteration 2, loss = 79.49531034
Iteration 3, loss = 77.34834501
Iteration 4, loss = 75.35050313
Iteration 5, loss = 73.46520967
Iteration 6, loss = 71.63307816
Iteration 7, loss = 69.85939805
Iteration 8, loss = 68.06979524
Iteration 9, loss = 66.27205647
Iteration 10, loss = 64.44518529
Iteration 11, loss = 62.58203927
Iteration 12, loss = 60.68473452
Iteration 13, loss = 58.75728885
Iteration 14, loss = 56.76614898
Iteration 15, loss = 54.76682713
Iteration 16, loss = 52.73234438
Iteration 17, loss = 50.68801661
Iteration 18, loss = 48.63945983
Iteration 19, loss = 46.57401604
Iteration 20, loss = 44.50109380
Iteration 21, loss = 42.43525102
Iteration 22, loss = 40.38631215
Iteration 23, loss = 38.38087392
Iteration 24, loss = 36.42181257
Iteration 25, loss = 34.49945545
Iteration 26, loss = 32.67858808
Iteration 27, loss = 30.91417343
Iteration 28, loss = 29.24413838
Iteration 29, loss = 27.62798099
Iteration 30, loss = 26.11567685
Iteration 31, loss 

Iteration 271, loss = 0.59461818
Iteration 272, loss = 0.59153317
Iteration 273, loss = 0.58867560
Iteration 274, loss = 0.58429151
Iteration 275, loss = 0.58105603
Iteration 276, loss = 0.57783670
Iteration 277, loss = 0.57442553
Iteration 278, loss = 0.57085685
Iteration 279, loss = 0.56766533
Iteration 280, loss = 0.56440029
Iteration 281, loss = 0.56071701
Iteration 282, loss = 0.55779235
Iteration 283, loss = 0.55450933
Iteration 284, loss = 0.55139620
Iteration 285, loss = 0.54812785
Iteration 286, loss = 0.54528715
Iteration 287, loss = 0.54192486
Iteration 288, loss = 0.53925910
Iteration 289, loss = 0.53544626
Iteration 290, loss = 0.53297949
Iteration 291, loss = 0.52965600
Iteration 292, loss = 0.52729721
Iteration 293, loss = 0.52346857
Iteration 294, loss = 0.52125097
Iteration 295, loss = 0.51782573
Iteration 296, loss = 0.51524915
Iteration 297, loss = 0.51218413
Iteration 298, loss = 0.50928271
Iteration 299, loss = 0.50696947
Iteration 300, loss = 0.50391259
Iteration 

Iteration 547, loss = 0.15868317
Iteration 548, loss = 0.15790882
Iteration 549, loss = 0.15727149
Iteration 550, loss = 0.15643466
Iteration 551, loss = 0.15594430
Iteration 552, loss = 0.15519001
Iteration 553, loss = 0.15461146
Iteration 554, loss = 0.15394677
Iteration 555, loss = 0.15336543
Iteration 556, loss = 0.15266964
Iteration 557, loss = 0.15206450
Iteration 558, loss = 0.15145149
Iteration 559, loss = 0.15097065
Iteration 560, loss = 0.15019139
Iteration 561, loss = 0.14973523
Iteration 562, loss = 0.14944518
Iteration 563, loss = 0.14890146
Iteration 564, loss = 0.14844712
Iteration 565, loss = 0.14747234
Iteration 566, loss = 0.14711231
Iteration 567, loss = 0.14676276
Iteration 568, loss = 0.14632942
Iteration 569, loss = 0.14555197
Iteration 570, loss = 0.14461291
Iteration 571, loss = 0.14412852
Iteration 572, loss = 0.14413238
Iteration 573, loss = 0.14312710
Iteration 574, loss = 0.14259248
Iteration 575, loss = 0.14179720
Iteration 576, loss = 0.14116295
Iteration 

Iteration 821, loss = 0.05791351
Iteration 822, loss = 0.05798195
Iteration 823, loss = 0.05778387
Iteration 824, loss = 0.05742382
Iteration 825, loss = 0.05750365
Iteration 826, loss = 0.05733375
Iteration 827, loss = 0.05699352
Iteration 828, loss = 0.05667991
Iteration 829, loss = 0.05666746
Iteration 830, loss = 0.05640967
Iteration 831, loss = 0.05615744
Iteration 832, loss = 0.05600113
Iteration 833, loss = 0.05583245
Iteration 834, loss = 0.05639518
Iteration 835, loss = 0.05579455
Iteration 836, loss = 0.05563308
Iteration 837, loss = 0.05517208
Iteration 838, loss = 0.05485853
Iteration 839, loss = 0.05479621
Iteration 840, loss = 0.05467494
Iteration 841, loss = 0.05429510
Iteration 842, loss = 0.05437705
Iteration 843, loss = 0.05397839
Iteration 844, loss = 0.05368119
Iteration 845, loss = 0.05358052
Iteration 846, loss = 0.05341041
Iteration 847, loss = 0.05324670
Iteration 848, loss = 0.05319572
Iteration 849, loss = 0.05302404
Iteration 850, loss = 0.05288047
Iteration 

Iteration 1100, loss = 0.02776196
Iteration 1101, loss = 0.02780486
Iteration 1102, loss = 0.02775136
Iteration 1103, loss = 0.02764069
Iteration 1104, loss = 0.02767646
Iteration 1105, loss = 0.02755198
Iteration 1106, loss = 0.02747685
Iteration 1107, loss = 0.02762615
Iteration 1108, loss = 0.02750547
Iteration 1109, loss = 0.02741312
Iteration 1110, loss = 0.02743990
Iteration 1111, loss = 0.02762753
Iteration 1112, loss = 0.02751122
Iteration 1113, loss = 0.02734395
Iteration 1114, loss = 0.02730134
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPRegressor(alpha=1, hidden_layer_sizes=16, learning_rate='adaptive',
             max_iter=4000, verbose=True)

In [11]:
test = pd.read_csv("test.csv")


temp = {"Id":[],"SalePrice":[]}

test_x = test
# encoding cat data
test_x = test_x.replace(replace_dir)
# nan remove
test_x = test_x.fillna(0)


# get relative feature
test_x = test_x[train_x_label]
# oulier dealing
test_x = numData_outlier_limit(test_x,"",3)
test_x  = z_score(test_x)
#print(test_x)




#print(num_data_test)

#test_x  = log(num_data_test).replace([np.inf, -np.inf], np.nan).fillna(0)


test_y_hat = reg.predict(test_x)

count = 0
for i in test_y_hat:
    
    temp["Id"].append(test["Id"][count])
    
    #temp["SalePrice"].append(i**2)
    temp["SalePrice"].append(np.exp(i))
    count+=1

    
#print(temp)
#i = np.random.randint(0,len(temp["Id"]))
i=0
print("第",i,"筆",temp["Id"][i],temp["SalePrice"][i])
temp = pd.DataFrame(temp)

#for i in range(len(temp["Id"])):
#    print("第一筆",temp["Id"][i],temp["SalePrice"][i])



#test_y 
#mean_squared_error()

第 0 筆 1461 109552.51027709042


In [12]:
train_y_hat = reg.predict(train_x)

for i in range(len(train_y_hat)):
    #train_y_hat[i]=(train_y_hat[i])**2
    train_y_hat[i]=np.exp(train_y_hat[i])
for i in range(len(test_y_hat)):
    #test_y_hat[i]=(test_y_hat[i])**2
    test_y_hat[i]=np.exp(test_y_hat[i])    

# validation variance error (overfitting) or bias error (underfiting)
from sklearn.metrics import mean_squared_log_error
train_err = mean_squared_log_error(train_y , train_y_hat)





test_y = pd.read_csv("solution.csv")



test_err = mean_squared_log_error(test_y["SalePrice"] , test_y_hat)

print("training error:",train_err)
print("testing error :",test_err)

training error: 0.012637477194453687
testing error : 0.019826672626407537


In [7]:
!del answer.csv
temp.to_csv("answer.csv",index=False)

In [30]:
#saving and loading part 

import joblib
joblib.dump(reg,'svm_0.41.joblib')
#reg2 =joblib.load("NNmodel_loss_0.18.pkl")
#reg2.predict(num_data_test)[0]

['svm_0.41.joblib']

In [13]:
#ensemble
import sklearn
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.neural_network import MLPRegressor
#reg = sklearn.neural_network.MLPRegressor(max_iter=15000)
from sklearn import svm
from sklearn.ensemble import VotingRegressor
from sklearn import tree

from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor(max_depth=4, random_state=0)

from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=75, learning_rate=0.01,\
max_depth=1, random_state=0, loss='ls')

#tree = tree.DecisionTreeRegressor()
lr = linear_model.LinearRegression()
lr_ridge  = Ridge(alpha=0.2)
lr_bay  = linear_model.ARDRegression() 
#svm = svm.SVR()
nn = sklearn.neural_network.MLPRegressor(hidden_layer_sizes=16,max_iter=4000,alpha=1,verbose=True ,learning_rate="adaptive")
ereg = VotingRegressor(estimators=[('nn',nn),('lr_ridge',lr_ridge),('lr_bay',lr_bay),("forest",forest) ,('lr',lr),('gbr',gbr)])
ereg = ereg.fit(train_x, np.log(train_y))

test_y_hat = ereg.predict(test_x)

temp = {"Id":[],"SalePrice":[]}
count = 0

for i in test_y_hat:
    
    temp["Id"].append(test["Id"][count])
    
    #temp["SalePrice"].append((i)**2)
    temp["SalePrice"].append(np.exp(i))
    count+=1

    
#print(temp)
print("第一筆",temp["Id"][0],temp["SalePrice"][0])
temp = pd.DataFrame(temp)
!del answer.csv
temp.to_csv("answer.csv",index=False)

train_y_hat = ereg.predict(train_x)

for i in range(len(train_y_hat)):
    #train_y_hat[i]=(train_y_hat[i])**2
    train_y_hat[i]=np.exp(train_y_hat[i])
for i in range(len(test_y_hat)):
    #test_y_hat[i]=(test_y_hat[i])**2
    test_y_hat[i]=np.exp(test_y_hat[i])    

# validation variance error (overfitting) or bias error (underfiting)
from sklearn.metrics import mean_squared_log_error
train_err = mean_squared_log_error(train_y , train_y_hat)


    


test_y = pd.read_csv("solution.csv")



test_err = mean_squared_log_error(test_y["SalePrice"] , test_y_hat)

print("training error:",train_err)
print("testing error :",test_err)

for e in ereg.transform(test_x.iloc[:1,:]):
    print(np.exp(e))

Iteration 1, loss = 74.82719841
Iteration 2, loss = 72.01899433
Iteration 3, loss = 69.34653120
Iteration 4, loss = 66.70599508
Iteration 5, loss = 64.11852040
Iteration 6, loss = 61.48711662
Iteration 7, loss = 58.81137470
Iteration 8, loss = 56.09484504
Iteration 9, loss = 53.32096839
Iteration 10, loss = 50.56096677
Iteration 11, loss = 47.79378256
Iteration 12, loss = 45.06811012
Iteration 13, loss = 42.35821404
Iteration 14, loss = 39.75396374
Iteration 15, loss = 37.27028095
Iteration 16, loss = 34.86755158
Iteration 17, loss = 32.62596856
Iteration 18, loss = 30.52638205
Iteration 19, loss = 28.60090619
Iteration 20, loss = 26.83177905
Iteration 21, loss = 25.21886054
Iteration 22, loss = 23.76460244
Iteration 23, loss = 22.46215637
Iteration 24, loss = 21.29501036
Iteration 25, loss = 20.25184723
Iteration 26, loss = 19.33466433
Iteration 27, loss = 18.51261344
Iteration 28, loss = 17.78302104
Iteration 29, loss = 17.10166072
Iteration 30, loss = 16.49986057
Iteration 31, loss 

Iteration 277, loss = 0.65636536
Iteration 278, loss = 0.65238369
Iteration 279, loss = 0.64816207
Iteration 280, loss = 0.64408544
Iteration 281, loss = 0.63997571
Iteration 282, loss = 0.63698734
Iteration 283, loss = 0.63264257
Iteration 284, loss = 0.62845680
Iteration 285, loss = 0.62518638
Iteration 286, loss = 0.62176375
Iteration 287, loss = 0.61728893
Iteration 288, loss = 0.61359713
Iteration 289, loss = 0.60980101
Iteration 290, loss = 0.60609691
Iteration 291, loss = 0.60224430
Iteration 292, loss = 0.59827493
Iteration 293, loss = 0.59470726
Iteration 294, loss = 0.59113984
Iteration 295, loss = 0.58762948
Iteration 296, loss = 0.58392783
Iteration 297, loss = 0.58084652
Iteration 298, loss = 0.57698977
Iteration 299, loss = 0.57386018
Iteration 300, loss = 0.56986919
Iteration 301, loss = 0.56631052
Iteration 302, loss = 0.56276710
Iteration 303, loss = 0.55901873
Iteration 304, loss = 0.55583013
Iteration 305, loss = 0.55256041
Iteration 306, loss = 0.54888528
Iteration 

Iteration 548, loss = 0.17572832
Iteration 549, loss = 0.17507299
Iteration 550, loss = 0.17442004
Iteration 551, loss = 0.17367796
Iteration 552, loss = 0.17319213
Iteration 553, loss = 0.17250006
Iteration 554, loss = 0.17184115
Iteration 555, loss = 0.17123637
Iteration 556, loss = 0.17045965
Iteration 557, loss = 0.17002081
Iteration 558, loss = 0.16953168
Iteration 559, loss = 0.16865297
Iteration 560, loss = 0.16813132
Iteration 561, loss = 0.16771376
Iteration 562, loss = 0.16681315
Iteration 563, loss = 0.16629096
Iteration 564, loss = 0.16569610
Iteration 565, loss = 0.16504176
Iteration 566, loss = 0.16453908
Iteration 567, loss = 0.16392356
Iteration 568, loss = 0.16378897
Iteration 569, loss = 0.16288947
Iteration 570, loss = 0.16232869
Iteration 571, loss = 0.16176144
Iteration 572, loss = 0.16109327
Iteration 573, loss = 0.16061660
Iteration 574, loss = 0.16009189
Iteration 575, loss = 0.15951214
Iteration 576, loss = 0.15888804
Iteration 577, loss = 0.15843383
Iteration 

Iteration 819, loss = 0.07488987
Iteration 820, loss = 0.07474633
Iteration 821, loss = 0.07440819
Iteration 822, loss = 0.07422782
Iteration 823, loss = 0.07408853
Iteration 824, loss = 0.07409674
Iteration 825, loss = 0.07392553
Iteration 826, loss = 0.07348077
Iteration 827, loss = 0.07317821
Iteration 828, loss = 0.07284249
Iteration 829, loss = 0.07269374
Iteration 830, loss = 0.07243344
Iteration 831, loss = 0.07234775
Iteration 832, loss = 0.07194165
Iteration 833, loss = 0.07190445
Iteration 834, loss = 0.07152284
Iteration 835, loss = 0.07139925
Iteration 836, loss = 0.07119508
Iteration 837, loss = 0.07090537
Iteration 838, loss = 0.07053906
Iteration 839, loss = 0.07063092
Iteration 840, loss = 0.07056392
Iteration 841, loss = 0.07016217
Iteration 842, loss = 0.06980594
Iteration 843, loss = 0.06965493
Iteration 844, loss = 0.06946470
Iteration 845, loss = 0.06924405
Iteration 846, loss = 0.06905174
Iteration 847, loss = 0.06846538
Iteration 848, loss = 0.06827344
Iteration 

Iteration 1092, loss = 0.03468440
Iteration 1093, loss = 0.03484436
Iteration 1094, loss = 0.03495529
Iteration 1095, loss = 0.03459723
Iteration 1096, loss = 0.03441484
Iteration 1097, loss = 0.03437441
Iteration 1098, loss = 0.03503181
Iteration 1099, loss = 0.03490815
Iteration 1100, loss = 0.03455174
Iteration 1101, loss = 0.03407825
Iteration 1102, loss = 0.03406931
Iteration 1103, loss = 0.03388587
Iteration 1104, loss = 0.03386530
Iteration 1105, loss = 0.03372670
Iteration 1106, loss = 0.03376765
Iteration 1107, loss = 0.03383093
Iteration 1108, loss = 0.03384731
Iteration 1109, loss = 0.03356486
Iteration 1110, loss = 0.03350900
Iteration 1111, loss = 0.03398624
Iteration 1112, loss = 0.03351631
Iteration 1113, loss = 0.03349544
Iteration 1114, loss = 0.03334213
Iteration 1115, loss = 0.03310620
Iteration 1116, loss = 0.03312184
Iteration 1117, loss = 0.03311249
Iteration 1118, loss = 0.03300973
Iteration 1119, loss = 0.03311623
Iteration 1120, loss = 0.03299932
Iteration 1121

In [4]:
import sklearn


print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.1.
